# 2주차 부트 캠프

## 1. Epsilon

In [1]:
import sys

# 64bit 컴퓨터에서 정확히 표현 가능한 자릿수 확인
sys.float_info.dig

15

In [2]:
num = 2.0**53
num

9007199254740992.0

In [3]:
# num는 16자리 숫자. 즉, 마지막 숫자는 불확실함.
# 64bit 컴퓨터는 15자리까지 숫자의 정확성을 보장받음
# 이는 실수 구현 당시 속도를 택하고 정확도를 버린 결과
len(str(int(num)))

16

In [4]:
# 2진수 53자리까지 완벽하게 표현이 가능
# 10진수는 15자리까지 완벽하게 표현이 가능
# 1.0을 십진수로 표현하고 싶다!!
# 1.000000000000.......0_(2) * 2^0 -> 1.0 # representable number
# 이 다음에 표현할 수 있는 수는?
# 1.000000000000.......1_(2) * 2^0
# 위의 두 수 사이의 차이 == epsilon

# sys module에서 epsilon 받아오기
ep = sys.float_info.epsilon

In [5]:
print("epsilon : {}".format(ep))
print("this number means, 2.220...*10^-16")

epsilon : 2.220446049250313e-16
this number means, 2.220...*10^-16


In [6]:
# 64bit에서 11개의 exp term, 부호 표시 1개 제외 52자리 표현가능
2**-52

2.220446049250313e-16

## absolute comparison

In [7]:
a = 0.1 * 3
b = 0.3

In [8]:
# 육안으로 보면 같은 결과.
# 하지만 결과는?
a==b

False

In [9]:
a

0.30000000000000004

In [10]:
b

0.3

In [11]:
# for 게임 개발하시는 분들의 입장,
# 실수 쓸 때 == 쓰지마라!
# 무조건 >=, <=
# 게임에서는 double은 안쓰고 float를 씀
# 실수가 같은지 비교해야할 상황이 온다면 어떻게 해결할 것인가?

# data science, coding
if a==b:
    print("thing")
else:
    print("fantastic baby")

fantastic baby


## 절대 비교
#### absolute comparison
  - interface
  - is_equal(a,b) -> boolean
  - a와 b의 차이를 절대값으로 만듦
  - 이 값  1e-10 >> 1*10^-10
  - diff < upper than True

In [12]:
print("a-b is {}".format(a-b),"\nepsilon is {}".format(ep))

a-b is 5.551115123125783e-17 
epsilon is 2.220446049250313e-16


In [13]:
ep > a-b

True

In [14]:
# 다음과 같은 표현으로도 python은 숫자 표현이 가능하다!
c = 1e10
c

10000000000.0

In [15]:
from math import fabs
print("abs(a-b) is {}".format(abs(a-b)))
print("fabs(a-b) is {}".format(fabs(a-b)))
print("is this same? : {}".format(abs(a-b)==fabs(a-b)))

abs(a-b) is 5.551115123125783e-17
fabs(a-b) is 5.551115123125783e-17
is this same? : True


In [16]:
# Since 절대비교, 특정한 값과 비교
def is_equal_by_absolute(a,b):
    return fabs(a-b) <=1e-10

a = 0.1 * 3
b = 0.3
if a==b:
    print("wow")
elif is_equal_by_absolute(a,b):
    print("Great")
else:
    print("no")

Great


## Relative comparison(상대 비교)

In [17]:
# 값을 고정시켜 놓으면 일반화시킬 수 없다
# 즉, 두 실수의 scale로 조정하여 변화율을 측정,
# 들어오는 input value에 맞춰서 error rate를 측정한다.

def is_equal_by_relative(a,b):
    diff = fabs(a-b)
    maximal = max(fabs(a),fabs(b))
    rel_error = diff / maximal
    return rel_error <= 1e-8

if is_equal_by_relative(a,b):
    print("Great")

Great


In [18]:
# 다음은 절대 참조와 상대 참조 모두를 활용하는 함수이다.

def is_equal(a,b):
    diff = fabs(a-b)
    rel_error = diff / max(fabs(a),fabs(b))
    if diff <= 1e-10:
        result = True
    elif rel_error <= 1e-8:
        result = True
    else:
        result = False
    return result

if is_equal(a,b):
    print("thing")

thing


In [19]:
# is_equal함수의 다른 표현 방법이다.

def is_equal2(a,b):
    diff = fabs(a-b)
    rel_error = diff / max(fabs(a),fabs(b))
    if diff <= 1e-10:
        return True
    return rel_error <= 1e-8

In [20]:
# 위의 is_equal함수에서 상수를 ep으로 활용한 함수이다.

def is_equal_by_epsilon(a,b):
    ep = sys.float_info.epsilon
    diff = fabs(a-b)
    rel_error = diff / max(fabs(a),fabs(b))
    if diff <= ep:
        return True
    return rel_error <= ep

In [21]:
# 단순하게 epsilon이랑 비교할 경우, 다음과 같은 문제점이 발생할 수 있다.

a = 0.01
s = 0.0
for _ in range(100):
    s+=a
s
# 우리가 기대하기로, s는 1이 되어야 할 것인데,

1.0000000000000007

In [22]:
t = 1.0
is_equal_by_epsilon(t,s)
# 우리가 제작한 함수로는 위 두 수를 "같다"라고 표현할 수가 없다.

False

In [23]:
# 즉, 가중치를 두어 우리가 조절할 수 있게 만들자!

def is_equal_by_epsilon_weighted(a,b, w=0):
    ep = sys.float_info.epsilon
    diff = fabs(a-b)
    rel_error = diff / max(fabs(a),fabs(b))
    #if diff <= ep:
    #    return True
    return rel_error <= ep*(2**w)

if is_equal_by_epsilon_weighted(t,s,w=2):
    print("thing")

thing


## 2. Firstclass_function
  - 함수를
  - 첫째, argument (인자)로 전달할 수 있는가
  - 둘째, variable (변수)로 전달할 수 있는가
  - 셋째, return (반환값)으로 함수를 사용할 수 있는가
  - 위의 세 조건을 만족하는 언어를 Firstclass_function을 만족한다고 한다.
- python은 이것을 지원하는 언어
- c언어는 포인터를 넘겨서 사용
- js는 지원을 함

1. argument

In [24]:
def f(a,b):
    return a+b

# 함수를 argument로 calling
# 이게 먹힌다는 얘기는 함수를 인자로 전달할 수 있다는 얘기
def g(func, a, b):
    return func(a,b) 

# 보는 바와 같이 잘 먹힌다.
g(f,1,2)

3

2. variable

In [25]:
f_var = f
f_var(1,2) # 함수를 변수에 저장할 수 있다.

3

3. return

In [26]:
def calc(kind):
    if kind == "add":
        def add(a,b):
            return a+b
        return add
    elif kind == "subtract":
        def subtract(a,b):
            return a-b
        return subtract
    
adder = calc("add")
# 함수를 return값으로 받는 것이 가능하다.
type(adder)

function

In [27]:
adder(1,2)

3

## 3. Stackframe

In [28]:
# 파이썬 특화
a = 10 # 전역 변수
def func():
    a = 20 # 함수 내에 선언된 변수 : 지역 변수
    print("함수 내에서 선언된 a :", a)
func()
print("함수 밖에서 선언된 a :", a)

함수 내에서 선언된 a : 20
함수 밖에서 선언된 a : 10


In [29]:
# 파이썬 특화
a = 10
def func():
    global a
    a = 20
    print("함수 내에서 선언된 a :", a)
    
# 위의 다른 것을 해결하기 위해선 global을 선언해주면 된다.
func()
print("함수 밖에서 선언된 a :", a)

함수 내에서 선언된 a : 20
함수 밖에서 선언된 a : 20


- Nonlocal

In [30]:
a = 10
def outer():
    b = 20 # in python, free variable for inner
    def inner():
        b = 45
        print("{} in inner".format(b))
    inner()
    print("{} in outer".format(b))
outer()

# outer의 b는 global도 아니고 local도 아니고... 도대체 무엇일까?

45 in inner
20 in outer


In [31]:
a = 10
def outer():
    b = 20 # in python, free variable for inner
    def inner():
        nonlocal b
        b = 45
        print("{} in inner".format(b))
    inner()
    print("{} in outer".format(b))
outer()

45 in inner
45 in outer


### 다시 제대로 StackFrame

In [32]:
# 전역변수 vs 지역변수
c = 10 # 전역 변수
def func(a, b):
    c = 20 # 함수 내에 선언된 변수 : 지역 변수
    return a + b
d = func(10, 20)

# global에 c = 10 쌓임
# d = 를 쌓는 순간 함수라는 stackframe이 쌓임
# stackframe이 쌓이며,
#    - 밑에서 부터 하나하나 쌓아올림
#    - b = 20
#    - a = 10
#    - c = 20
# global의 c랑 stackframe의 c는 완전 다른 memory 공간에 존재함
# func()를 호출하는 순간 stackframe은 소멸됨.
# 때문에 메모리적인 관점에서 global c = 10은 전혀 바뀌지 않음
# func() stackframe이 쌓였다가 return함과 동시에 할당하고 memory에서 버림

http://pythontutor.com/

- 위의 사이트 참고, 내가 짠 코드가 어떻게 메모리 상에서 처리되는지 시각화 자료를 볼 수 있다.

### stack overflow

In [33]:
def func():
    a = 10
    func()
func()

RecursionError: maximum recursion depth exceeded

In [34]:
# func()호출하면 function frame을 하나 만듬
# 그 안에서 또 func()호출, 또 frame을 만듬
# 계속 반복
# 함수가 호출될 때 마다 stackframe이 하나씩 만듬
# 그러다가 stack 용량이 넘침
# 이것을 RecursionError을 일으킴
# 사실은 python은 stack을 직접적으로 사용할 수가 없다.

# stack을 쌓다가 터진 것을 stack overflow 라고 함

# http://pythontutor.com

cnt=0
def f(m=5):
    global cnt
    cnt += 1
    if cnt>m:
        return
    a=10
    f()
    
f(m=5)
# 이걸로 시험해보시길

## 4. Closure
  - 위의 firstclass function이 되어야 만들 수 있음
  - java script에서도 큰 역할을 하고 있음
  - why 쓸 수 밖에 없나?
  - js에는 class가 없음. 일부로 구현을 안함.
  - js 진영 : 우리도 OOP 쓰고 싶어요!
  - oop의 대용품으로 나온 것이 closure!
  - definition Clousure
  - 함수 내부에 상태 정보를 가지고 있음
  - python에서는 closure를 쓰실 일이 거의 없음..
  - becuase OOP로 대부분의 것들이 해결 가능!

In [35]:
# 지금부터 짜는 코드는 oop로 짜면 단번에 해결됨
# 함수 -> 계좌
def account(name, balance):
    def transaction(money):
        nonlocal balance
        balance += money
        return name, balance
    return transaction
# 이것이 closure

# 내부 정보를 남겨두고 조회를 하는 것이 closure

greg_acnt = account("Greg", 50000000)
mark_acnt = account("Mark", 3000000)

print("type(greg_acnt) : {}".format(type(greg_acnt)))
print("type(mark_acnt) : {}".format(type(mark_acnt)))

type(greg_acnt) : <class 'function'>
type(mark_acnt) : <class 'function'>


In [36]:
print("greg_acnt(5000000) : {}".format(greg_acnt(5000000)))
print("mark_acnt(5000000) : {}".format(mark_acnt(5000000)))

greg_acnt(5000000) : ('Greg', 55000000)
mark_acnt(5000000) : ('Mark', 8000000)


In [37]:
print("greg_acnt.__name__ : {}".format(greg_acnt.__name__))
print("mark_acnt.__name__ : {}".format(mark_acnt.__name__))

# closure를 이해하고 decorator를 만들어보자!
# 그전에 이해도를 높이고!

greg_acnt.__name__ : transaction
mark_acnt.__name__ : transaction


## 5. Decorator

In [38]:
class A:
    def __init__(self, a):
        self.__a = a
#    @func2
#    @func1
    @property
    def a(self):
        return self.__a
    
    @a.setter
    def a(self, new_a):
        self.__a = new_a
        
# decorator는 어떤 함수에 기능을 추가할 때 사용함

In [39]:
def func(a, b):
    return a+b

func(1,2)

3

In [40]:
def func(a, b):
    return a-b

func(1,2)

-1

- 위의 함수를 덮어버림

- 이제부터 설명을 시작합니다~

- 잠시 asterisk에 대해 설명

In [42]:
# unpacking
li = [1,2,3,4,5]
a, *b = li
print(a, b)

1 [2, 3, 4, 5]


In [43]:
def func(*args, **kwargs):
    print(args) # tuple로 묶임
    print(kwargs) # dictionary로 묶임
    
# 함수를 선언할 때의 argument의 의미는, 받겠다.
# 함수를 호출할 때의 이 의미는, 풀겠다.

In [44]:
func(1,2,3,{"a":1,"b":2})

(1, 2, 3, {'a': 1, 'b': 2})
{}


In [45]:
func(1,2,3,a=4,b=5)

(1, 2, 3)
{'a': 4, 'b': 5}


In [46]:
tu = (1,2,3)
dic = {"a":4, "b":5}

In [47]:
func(tu, dic)

((1, 2, 3), {'a': 4, 'b': 5})
{}


In [48]:
func(*tu, *dic)

(1, 2, 3, 'a', 'b')
{}


In [49]:
func(*tu, **dic)

(1, 2, 3)
{'a': 4, 'b': 5}


- 하던 것을 마져 해 볼까요?

In [56]:
def outer(org_func):
    def inner(*args, **kwargs):
        # 필요한 기능 추가
        print("things to do")
        return org_func(*args, **kwargs) # 함수니까 당연히 callable 가능
    return inner

def add(a,b):
    return a+b

add(1,2)

3

In [57]:
add = outer(add)
add

<function __main__.outer.<locals>.inner(*args, **kwargs)>

In [58]:
add(1,2)

things to do


3

In [59]:
@outer
def add(a,b):
    return a+b

In [60]:
add(1,2)

things to do


3

In [61]:
## decorator는 기능을 추가하는 것, 붙이는 것

add.__name__
# outer 함수의 결과값은 inner

# 이거 맘에 안들어! outer 안에 있는 함수가 뭔지 알고?

'inner'

In [62]:
from functools import wraps

def outer(org_func):
    @wraps(org_func)
    def inner(*args, **kwargs):
        # 필요한 기능 추가
        print("things to do")
        return org_func(*args, **kwargs) # 함수니까 당연히 callable 가능
    return inner

@outer
def add(a,b):
    return a+b

In [63]:
add(1,2)

things to do


3

In [64]:
add.__name__

'add'

### Benchmarker
  - 함수 실행 시간을 비교하여 성능 파악
  - 함수 실행 시간을 알려주는 decorator를 만들 것임

In [65]:
from functools import wraps
import time

def benchmarker(org_func):
    @wraps(org_func)
    def inner(*args, **kwargs):
        #epoch 1911.~ 시간을 정수형태 초단위로 반환
        start = time.time()
        # 이러면 치명적인 오류 존재! return으로 함수가 끝나버림..
        return org_func(*args, **kwargs)
        elapsed = time.time() - start
        print("elapsed time of {} : {}".format(org_func.__name__, elapsed))
    return inner

del benchmarker

def benchmarker(org_func):
    @wraps(org_func)
    def inner(*args, **kwargs):
        start = time.time()
        result = org_func(*args, **kwargs)
        elapsed = time.time() - start
        print("elapsed time of {} : {}".format(org_func.__name__, elapsed))
        return result
    return inner

In [66]:
@benchmarker
def add(a,b):
    time.sleep(3) # 초 단위로 셈
    return a+b

In [67]:
add(1,2)

elapsed time of add : 3.000967025756836


3

In [68]:
# ronud()함수를 사용, 깔끔하게 만들어보자.

def benchmarker(org_func):
    @wraps(org_func)
    def inner(*args, **kwargs):
        start = time.time() #epoch 1911.~ 시간을 정수형태 초단위로 반환
        result = org_func(*args, **kwargs)
        elapsed = time.time() - start
        print("elapsed time of {} : {}".format(org_func.__name__, round(elapsed, 6)))
        return result
    return inner

In [69]:
@benchmarker
def add(a,b):
    time.sleep(3) # 초 단위로 쉼
    return a+b
add(1,2)

elapsed time of add : 3.000112


3

### senior, architechter >>> decorator 작동 원리를 아는 것 필요